# Characterizing the system

In [Part 1](../../notebooks/workflow/workflow01.md) of this series we described the physical system, including an idealized physics model:

```{image} _static/dc_motor.png
:class: only-light
```

```{image} _static/dc_motor_dark.png
:class: only-dark
```

$$
\begin{align*}
L \frac{di}{dt} &= V - iR - k_\mathcal{E} \omega \\
J \frac{d\omega}{dt} &= G k_\mathcal{E} i - b \omega.
\end{align*}
$$

Our first job is to see how well this model matches reality.

We will do this by collecting data from "step" and "ramp" response sequences using the real hardware and comparing to our idealized model.

This task is made somewhat difficult by the fact that while some parameters (like resistance $R$ and gear ratio $G$) are directly measurable or known from a datasheet, others (like the back EMF constant $k_\mathcal{E}$ or viscous friction $b$) are unknown.

Depending on how many such unknown parameters there are and how sensitive the model is to them, it may be feasible to simply manually tune the parameters until the model matches the data as well as possible.
A somewhat more automated approach is to sweep over parameters or perform a Monte Carlo search and simply pick the best of the candidate parameter sets.

Obviously, both approaches becomes increasingly difficult for more complex systems with large numbers of parameters.
Instead, we will use _parameter estimation_ to determine all the parameters at once as an optimization problem.
For more background, see the [tutorial](../sysid/parameter-estimation.ipynb) on parameter estimation in Archimedes.

In [1]:
# ruff: noqa: N802, N803, N806, N815, N816
import os

import matplotlib.pyplot as plt
import numpy as np

import archimedes as arc
from archimedes.docs.utils import extract_py_function, extract_py_class

THEME = os.environ.get("ARCHIMEDES_THEME", "dark")
arc.theme.set_theme(THEME)

## Data collection

The linearity of our model guarantees that very simple I/O data should suffice for fully characterizing the system.



## Model implementation

In [2]:
extract_py_class("motor.py", "MotorParams")

```python
@struct.pytree_node
class MotorParams:
    m: float  # Effective mass/inertia
    b: float  # Viscous friction
    L: float  # Motor inductance [H]
    R: float  # Motor resistance [Ohm]
    Kt: float  # Current -> torque scale [N-m/A]

    def asdict(self):
        return dataclasses.asdict(self)
```

In [3]:
extract_py_function("motor.py", "motor_ode")

```python
def motor_ode(
    t: float, x: np.ndarray, u: np.ndarray, params: MotorParams
) -> np.ndarray:
    params = params.asdict()

    i, _pos, vel = x
    (V,) = u
    m = params["m"]  # Effective mass/inertia
    b = params["b"]  # Viscous friction
    L = params["L"]  # Motor inductance
    Kt = params["Kt"]  # Current -> torque scale
    R = params["R"]

    Ke = Kt / GEAR_RATIO  # Velocity -> Back EMF scale

    i_t = (1 / L) * (V - (i * R) - Ke * vel)
    pos_t = vel
    vel_t = (1 / m) * (Kt * i - b * vel)

    return np.hstack([i_t, pos_t, vel_t])
```

## Parameter estimation